In [1]:
from openfisca_uk import PopulationSim
sim = PopulationSim()
import microdf as mdf
import numpy as np
import pandas as pd
%config Completer.use_jedi = False

In [49]:
h0 = sim.df(["household_net_income", "household_weight", "people_in_household"])
h = mdf.MicroDataFrame(h0, weights=h0.household_weight)
h["person_weight"] = h.household_weight * h.people_in_household

In [50]:
# h.household_net_income.sum() / 1e9
h.household_net_income.rank()


0        52321944.0
1        13893217.0
2        13892080.0
3        13890943.0
4         2908269.0
            ...    
43082    26041016.0
43083    26042064.0
43084    34358486.0
43085    25263728.0
43086    25264295.0
Length: 43087, dtype: float64

In [51]:
h.weights = h.person_weight
h.household_net_income.rank()

# h.household_net_income.sum() / 1e9

0        52321944.0
1        13893217.0
2        13892080.0
3        13890943.0
4         2908269.0
            ...    
43082    26041016.0
43083    26042064.0
43084    34358486.0
43085    25263728.0
43086    25264295.0
Length: 43087, dtype: float64

In [52]:
h.set_weights("person_weight")
h.household_net_income.rank()
# h.set_weights(h.household_weight)
# h.household_net_income.sum() / 1e9

0        52321944.0
1        13893217.0
2        13892080.0
3        13890943.0
4         2908269.0
            ...    
43082    26041016.0
43083    26042064.0
43084    34358486.0
43085    25263728.0
43086    25264295.0
Length: 43087, dtype: float64

In [54]:
h.set_weight_col("person_weight")
h.household_net_income.rank()

# h.household_net_income.sum() / 1e9

0        152479430.0
1         29887551.0
2         29884140.0
3         29880729.0
4          6727625.0
            ...     
43082     65200974.0
43083     65205166.0
43084     91660594.0
43085     62554225.0
43086     62555359.0
Length: 43087, dtype: float64

In [ ]:
h.weights = h.household_weight * h.people_in_household
h.equiv_household_net_income.sum() / 1e9

In [22]:
h.equiv_household_net_income.rank()

0        40605.0
1         7159.0
2         7159.0
3         7159.0
4         3422.0
          ...   
43082    16765.5
43083    16765.5
43084    35475.0
43085    21153.5
43086    21153.5
Name: equiv_household_net_income, Length: 43087, dtype: float64

In [17]:
159**2 * 52

1314612

In [5]:
d = sim.df(
    [
        "in_poverty_bhc",
        "equiv_household_net_income",
        "household_net_income",
        "poverty_line_bhc",
        "household_weight",
        "in_deep_poverty_bhc",
        "age"
    ], map_to="person"
)
md = mdf.MicroDataFrame(d, weights="household_weight")

In [19]:
dict(zip(["a", "b"], [1, 2]))

{'a': 1, 'b': 2}

In [6]:
md["pov"] = md.household_net_income < md.poverty_line_bhc

In [7]:
md[md.age < 18].in_poverty_bhc.__class__

microdf.generic.MicroSeries

In [8]:
md["age_group"] = np.where(md.age < 18, "Under 18",
                           np.where(md.age < 65, "18 to 64", "Over 64"))

In [9]:
md.groupby("age_group").in_poverty_bhc.mean() / 52

age_group
18 to 64    0.126451
Over 64     0.168529
Under 18    0.186858
dtype: float64

In [10]:
u18 = pd.DataFrame(md[md.age < 18])
mdf.weighted_mean(u18, "in_poverty_bhc", "household_weight") / 52

0.18685826115113088

In [11]:
md[md.age < 18].in_deep_poverty_bhc.mean() / 52

TypeError: Axis must be specified when shapes of a and weights differ.

In [ ]:
(md.pov == md.in_poverty_bhc / 52).mean()